## H1B Visa

- 첫 세션에서 다루지 못한 내용들이 많습니다 
- [이곳](https://www.kaggle.com/nsharan/h-1b-visa/kernels)의 kaggle kernel을 참고해 문제를 풀어주세요
- EDA에 자주 쓰이지만 매우 기초적인 함수만 사용했습니다

### Attribute Information

- __CASE_STATUS__
    - The CASE_STATUS field denotes the status of the application after LCA processing. Certified applications are filed with USCIS for H-1B approval.
<br>
<br>
- EMPLOYER_NAME
    - Name of the employer submitting labor condition application.
<br>
<br>
- SOC_NAME
    - Occupational name associated with the SOC_CODE. SOC_CODE is the occupational code associated with the job being requested for temporary labor condition, as classified by the Standard Occupational Classification (SOC) System.
<br>
<br>
- JOB_TITLE
    - Title of the job.
<br>
<br>
- FULL_TIME_POSITION
    - Y = Full Time Position; N = Part Time Position.
<br>
<br>
- PREVAILING_WAGE
    - Prevailing Wage for the job being requested for temporary labor condition. The wage is listed at annual scale in USD. The prevailing wage for a job position is defined as the average wage paid to similarly employed workers in the requested occupation in the area of intended employment. The prevailing wage is based on the employer’s minimum requirements for the position.
<br>
<br>
- YEAR
    - Year in which the H-1B visa petition was filed.
<br>
<br>

# EDA

In [ ]:

import numpy as np
import matplotlib
import pandas as pd
# import seaborn as sns
# from subprocess import check_output
# from matplotlib.ticker import FuncFormatter

#### 1. 데이터 확인
- h1b_train.csv 파일을 df_train으로 불러오세요

In [ ]:
file_path="./data/"
file_name=file_path+"h1b_train.csv"
df_train = pd.read_csv(file_name)


- feature의 개수와 training example의 개수를 확인하세요

In [ ]:
# Your Code 

print('Number of entries:',df_train.shape )

- 각 feature의 null값 개수를 확인하세요

In [ ]:
# Your Code Here
print('Number of null:',df_train.isna().sum() )

#### 2. 전처리

In [ ]:
# CASE_STATUS -> CERTIFIED or DENIED
# EMPLOYER_NAME -> 고용주 회사
# SOC_NAME -> 직업군
# JOB_TITLE -> 직무
df_train.head()

#### 2.1 CASE_STATUS
- Certified랑 Denied만 남기고 나머지 행은 지워주세요

In [ ]:
# Your Code Here
df_train =  df_train[(df_train['CASE_STATUS']=='CERTIFIED')|
                      (df_train['CASE_STATUS']=='DENIED') ]
print(df_train)

#### 2-2. Certified와 Denied의 개수를 확인하세요

In [ ]:
# Your Code Here
df_train['CASE_STATUS'].value_counts()

#### 2-3. Certified는 0, Denied는 1로 바꿔주세요

In [ ]:
# Your Code Here
df_train['CASE_STATUS']=df_train['CASE_STATUS'].replace('CERTIFIED',0).replace('DENIED',1)


In [ ]:
# DINIED 비율 = 3%
Cer_num=df_train['CASE_STATUS'].value_counts()[0]
Den_num=df_train['CASE_STATUS'].value_counts()[1]

Den_num/(Cer_num+Den_num)

#### 3-1. Full_time_position에서 Y는 1, N은 0으로 바꿔주세요

In [ ]:
# Your Code Here

df_train['FULL_TIME_POSITION']=df_train['FULL_TIME_POSITION'].replace('Y',0).replace('N',1)


#### 4-1. Prevailing_wage 변수를 히스토그램으로 그려보세요

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.subplots(1, 1, figsize=(12, 4))

plt.hist(df_train[df_train["PREVAILING_WAGE"]<200000].PREVAILING_WAGE.values,bins=200)

plt.xlim([0, 200000])
plt.show()

#### 4-2. Prevailing_wage 변수를 log 변환하고 다음과 같이 plot을 그려주세요

In [ ]:
# 방법1

# df_train['PREVAILING_WAGE'] = np.log(df_train['PREVAILING_WAGE'])
# print(df_train['PREVAILING_WAGE'])

# 방법2
import math
df_train['PREVAILING_WAGE']= df_train['PREVAILING_WAGE'].apply(lambda x : math.log(x) if x>0 else 0)
df_train['PREVAILING_WAGE']

In [ ]:
plt.subplots(1, 1, figsize=(12, 4))
# Your Code Here

plt.hist(df_train["PREVAILING_WAGE"].values,bins=500)

plt.xlim([9, 13])

#### 5.1 Employer_name

In [ ]:
# EMPLOYER_NAME을 보면 텍스트 전처리가 필요해 보입니다. 
# Punctuation이 문제인 것 같습니다.

df_train['EMPLOYER_NAME'].sort_values()

#### 5.2 EMPLOYER_NAME의 쉼표와 온점을 지워주세요

In [ ]:
df_train['EMPLOYER_NAME']=df_train['EMPLOYER_NAME'].str.replace('.','').str.replace(',','')
df_train['EMPLOYER_NAME']


#### 5.3 EMPLOYER_NAME 변수의 빈도를 이용해 다음과 같은 plot을 그려주세요

In [ ]:
grouped = df_train['EMPLOYER_NAME'].groupby(df_train['EMPLOYER_NAME']).size()
grouped.sort_values(ascending=False)[:20].plot(kind='barh')



#### 6.1 SOC_NAME 변수의 빈도를 이용해 다음과 같은 plot을 그려주세요

In [ ]:

df_train['SOC_NAME']=df_train['SOC_NAME'].apply(lambda x : str(x).upper())
grouped = df_train['SOC_NAME'].groupby(df_train['SOC_NAME']).size()
grouped.sort_values(ascending=False)[:20].plot(kind='barh')


#### 6.2 Year 변수를 이용해 연도별로 CASE_STATUS에 따라 빈도를 나타낸 plot을 그려주세요

In [ ]:

#  방법 1
# case_year  =  df_train.groupby(['YEAR','CASE_STATUS']).size().unstack()
# case_year.plot(kind="bar")
# fig = plt.gcf()
# fig.set_size_inches(15,6)
# plt.title('Case Status by Year')
# plt.show()
# case_year

#  방법 2
import seaborn as sns
case_year = df_train[['YEAR','CASE_STATUS','SOC_NAME']].groupby(['YEAR','CASE_STATUS']).count().reset_index()

sns.barplot(x="YEAR",y="SOC_NAME", hue="CASE_STATUS", data = case_year)
fig = plt.gcf()
fig.set_size_inches(15,6)
plt.title('Case Status by Year')
plt.show()
case_year


In [ ]:
# 연도에 따라 certificated 비율이 늘어나고 있습니다.

count_cer = case_year.loc[case_year['CASE_STATUS'] == 0]
count_de = case_year.loc[case_year['CASE_STATUS'] == 1]
rate = count_cer['SOC_NAME'].values / (count_cer['SOC_NAME'].values + count_de['SOC_NAME'].values)
rate

# 고생하셨습니다 :)